In [1]:
!pip install -r requirements.txt

%load_ext autoreload

In [65]:
# Imports
import os
import glob
import json
import numpy as np
import keras
from enum import Enum
from keras.models import Model, load_model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, BatchNormalization, Dropout
from keras.optimizers import Adam, RMSprop
from keras import backend as K
from random import randrange
import random
import math
import pypianoroll
from utils.midi_utils import play_midi, plot_pianoroll, get_music_metrics, process_pianoroll, process_midi
from constants import Constants
from augmentation import AddAndRemoveAPercentageOfNotes
from data_generator import PianoRollGenerator
from utils.generate_training_plots import GenerateTrainingPlots
from inference import Inference
from model import OptimizerType
from model import ArCnnModel

%autoreload

In [66]:
sampling_lower_bound_remove = 0 
sampling_upper_bound_remove = 100
sampling_lower_bound_add = 1
sampling_upper_bound_add = 3

In [67]:
# Customized loss function
class Loss():
    @staticmethod 
    def built_in_softmax_kl_loss(target, output):
        '''
        Custom Loss Function
        :param target: ground truth values
        :param output: predicted values
        :return kullback_leibler_divergence loss
        '''
        target = K.flatten(target)
        output = K.flatten(output)
        target = target / K.sum(target)
        output = K.softmax(output)
        return keras.losses.kullback_leibler_divergence(target, output)

In [70]:
#Import the MIDI files from the data_dir and save them with the midi_files variable  
midi_files = []
#midi_files.extend(glob.glob('data/gb/**/*.mid'))
#midi_files.extend(glob.glob('data/gb/tetris/*.mid'))
midi_files.extend(glob.glob('data/ms/**/*.mid'))
#midi_files.extend(glob.glob('data/master_system/*.mid'))
#midi_files.extend(glob.glob('data/nes/**/*.mid'))
#midi_files.extend(glob.glob('data/snes/**/*.mid'))

all_programs_used = []
all_voices_used = []
all_drums_used = []

def print_midi_info(midi_file):
    multi_track = pypianoroll.Multitrack()
    try:
        multi_track.parse_midi(midi_file,
                               algorithm='custom',
                               first_beat_time=0)
    except:
        print("midi file: {} is invalid. Ignoring during preprocessing".format(
            midi_file))
        pass
    
    multi_track.binarize()
    for track in multi_track.tracks:
        is_drum = ""
        if track.is_drum:
            drums_used = np.nonzero(np.sum(track.pianoroll, axis=0))[0]
            all_drums_used.extend(drums_used)
            is_drum = "[DRUM] " + str(drums_used)
        else:
            voices_used = np.nonzero(np.sum(track.pianoroll, axis=0))[0]
            all_voices_used.extend(voices_used)
            all_programs_used.append(track.program)            
        print("    {}: {} {}".format(track.program, track.name, is_drum))

# Generate MIDI file samples
def generate_samples(midi_files, bars, beats_per_bar, beat_resolution, bars_shifted_per_sample):
    """
    dataset_files: All files in the dataset
    return: piano roll samples sized to X bars
    """
    timesteps_per_nbars = bars * beats_per_bar * beat_resolution
    time_steps_shifted_per_sample = bars_shifted_per_sample * beats_per_bar * beat_resolution
    samples = []
    for midi_file in midi_files:
        print('process ' + midi_file + '...')
        print_midi_info(midi_file)
        pianoroll, drums = process_midi(midi_file, beat_resolution, Constants.program) # Parse the MIDI file and get the piano roll
        samples.extend(process_pianoroll(pianoroll, drums, time_steps_shifted_per_sample, timesteps_per_nbars))
    
    print('all programs used: ' + str(np.unique(all_programs_used, return_counts=True)))
    print('all voices used (min/max): {}/{}'.format(
        min(np.unique(all_voices_used)),
        max(np.unique(all_voices_used))))
    print('all drums used: ' + str(np.unique(all_drums_used, return_counts=True)))
    
    return samples

# Saving the generated samples into a dataset variable 
dataset_samples = generate_samples(midi_files, Constants.bars, Constants.beats_per_bar,Constants.beat_resolution, Constants.bars_shifted_per_sample)

# Shuffle the dataset
random.shuffle(dataset_samples);

dataset_size = len(dataset_samples)
dataset_split = math.floor(dataset_size * Constants.training_validation_split) 

training_samples = dataset_samples[0:dataset_split]
print("training samples length: {}".format(len(training_samples)))
validation_samples = dataset_samples[dataset_split + 1:dataset_size]
print("validation samples length: {}".format(len(validation_samples)))

process data/ms/jurassic_park/JPark-BadEnding.mid...
    80: Original composer: LOTTY & N.Gee 
    80: Sequenced by João *Johnnyz* Buaes 
    80: joaobuaes@zipmail.com.br 
    0:  [DRUM] [44]
process data/ms/jurassic_park/JPark-Stage4.mid...
    80: Original composer: LOTTY & N.Gee 
    80: Sequenced by João *Johnnyz* Buaes 
    80: joaobuaes@zipmail.com.br 
    0:  [DRUM] [38 44]
process data/ms/jurassic_park/JPark-Car1.mid...
    80: Original composer: LOTTY & N.Gee 
    80: Sequenced by João *Johnnyz* Buaes 
    80: joaobuaes@zipmail.com.br 
    0:  [DRUM] [35 38 41 44]
process data/ms/jurassic_park/JPark-Clear1.mid...
    80: Original composer: LOTTY & N.Gee 
    80: Sequenced by João *Johnnyz* Buaes 
    80: joaobuaes@zipmail.com.br 
    0:  [DRUM] [38 44]
process data/ms/jurassic_park/JPark-Ending.mid...
    80: Original composer: LOTTY & N.Gee 
    80: Sequenced by João *Johnnyz* Buaes 
    80: joaobuaes@zipmail.com.br 
    0:  [DRUM] [35 38 41 44]
process data/ms/jurassic_park/

    80: Original composer: ? 
    80: Sequenced by João *Johnnyz* Buaes 
    80: joaobuaes@zipmail.com.br 
    0:  [DRUM] [36 38]
process data/ms/alex_kidd/AKiMW_Janken.mid...
    80: Original composer: ? 
    80: Sequenced by João *Johnnyz* Buaes 
    80: joaobuaes@zipmail.com.br 
    0:  [DRUM] [36 38]
process data/ms/alex_kidd/AKiMW_Title.mid...
    80: Original composer: ? 
    80: Sequenced by João *Johnnyz* Buaes 
    80: joaobuaes@zipmail.com.br 
    0:  [DRUM] [36 38]
process data/ms/alex_kidd/AKiMW_Level.mid...
    80: Original composer: ? 
    80: Sequenced by João *Johnnyz* Buaes 
    80: joaobuaes@zipmail.com.br 
    0:  [DRUM] [36 38]
process data/ms/alex_kidd/AKiMW_Castle.mid...
    80: Original composer: ? 
    80: Sequenced by João *Johnnyz* Buaes 
    80: joaobuaes@zipmail.com.br 
    0:  [DRUM] [36 38]
process data/ms/alex_kidd/AKiMW_Peticopter.mid...
    80: Original composer: ? 
    80: Sequenced by João *Johnnyz* Buaes 
    80: joaobuaes@zipmail.com.br 
    0:  [DR

process data/ms/sonic_chaos/SonicChaos_SleepingEgg.mid...
    80: Original composer: Mix / Nagao N.Gee 
    80: Sequenced by João *Johnnyz* Buaes 
    80: joaobuaes@zipmail.com.br 
    0:  [DRUM] [36 42 46]
process data/ms/sonic_chaos/SonicChaos_Complete.mid...
    80: Original composer: Mix / Nagao N.Gee 
    80: Sequenced by João *Johnnyz* Buaes 
    80: joaobuaes@zipmail.com.br 
    0:  [DRUM] [36 42]
process data/ms/sonic_chaos/SonicChaos_ElectricEgg.mid...
    80: Original composer: Mix / Nagao N.Gee 
    80: Sequenced by João *Johnnyz* Buaes 
    80: joaobuaes@zipmail.com.br 
    0:  [DRUM] [36 42]
process data/ms/sonic_chaos/SonicChaos_Bonus.mid...
    80: Original composer: Mix / Nagao N.Gee 
    80: Sequenced by João *Johnnyz* Buaes 
    80: joaobuaes@zipmail.com.br 
    0:  [DRUM] [36 42]
process data/ms/sonic_chaos/SonicChaos-Unused.mid...
    80: Original composer: Mix / Nagao N.Gee 
    80: Sequenced by João *Johnnyz* Buaes 
    80: joaobuaes@zipmail.com.br 
    0:  [DRUM]

process data/ms/mickey/CoI_Complete.mid...
    80: Original composer: B. O 
    80: Sequenced by João *Johnnyz* Buaes 
    80: joaobuaes@zipmail.com.br 
midi file: data/ms/mickey/CoI_Complete.mid has no drum. Ignoring during preprocessing
process data/ms/mickey/CoI_Library.mid...
    80: Original composer: B. O 
    80: Sequenced by João *Johnnyz* Buaes 
    80: joaobuaes@zipmail.com.br 
midi file: data/ms/mickey/CoI_Library.mid has no drum. Ignoring during preprocessing
process data/ms/mickey/CoI_Victory.mid...
    80: Original composer: B. O 
    80: Sequenced by João *Johnnyz* Buaes 
    80: joaobuaes@zipmail.com.br 
midi file: data/ms/mickey/CoI_Victory.mid has no drum. Ignoring during preprocessing
process data/ms/mickey/CoI_Death.mid...
    80: Original composer: B. O 
    80: Sequenced by João *Johnnyz* Buaes 
    80: joaobuaes@zipmail.com.br 
    0:  [DRUM] [42]
process data/ms/mickey/CoI_Ending.mid...
    80: Original composer: B. O 
    80: Sequenced by João *Johnnyz* Buaes 


    80: Original composer: Sonic Team 
    80: Sequenced by João *Johnnyz* Buaes 
    80: joaobuaes@zipmail.com.br 
    0:  [DRUM] [38 42 55]
all programs used: (array([80]), array([515]))
all voices used (min/max): 0/110
all drums used: (array([35, 36, 38, 41, 42, 43, 44, 46, 49, 55, 69]), array([ 40,  45, 106,   2,  57,   1,  63,  24,   2,   1,   1]))
training samples length: 984
validation samples length: 109


In [71]:
# Piano Roll Input Dimensions
input_dim = (Constants.bars * Constants.beats_per_bar * Constants.beat_resolution, 
             Constants.number_of_pitches, 
             Constants.number_of_channels)
# Number of Filters In The Convolution
num_filters = 32
# Growth Rate Of Number Of Filters At Each Convolution
growth_factor = 2
# Number Of Encoder And Decoder Layers
num_layers = 5
# A List Of Dropout Values At Each Encoder Layer
dropout_rate_encoder = [0, 0.5, 0.5, 0.5, 0.5]
# A List Of Dropout Values At Each Decoder Layer
dropout_rate_decoder = [0.5, 0.5, 0.5, 0.5, 0]
# A List Of Flags To Ensure If batch_normalization Should be performed At Each Encoder
batch_norm_encoder = [True, True, True, True, False]
# A List Of Flags To Ensure If batch_normalization Should be performed At Each Decoder
batch_norm_decoder = [True, True, True, True, False]
# Path to Pretrained Model If You Want To Initialize Weights Of The Network With The Pretrained Model
pre_trained = False
# Learning Rate Of The Model
learning_rate = 0.001
# Optimizer To Use While Training The Model
optimizer_enum = OptimizerType.ADAM
# Batch Size
batch_size = 32
# Number Of Epochs
epochs = 2

In [72]:
# The Number of Batch Iterations Before A Training Epoch Is Considered Finished
steps_per_epoch = int(
    len(training_samples) * Constants.samples_per_ground_truth_data_item / int(batch_size))

print("The Total Number Of Steps Per Epoch Are: "+ str(steps_per_epoch))

The Total Number Of Steps Per Epoch Are: 246


In [73]:
## Training Data Generator
training_data_generator = PianoRollGenerator(sample_list=training_samples,
                                             sampling_lower_bound_remove = sampling_lower_bound_remove,
                                             sampling_upper_bound_remove = sampling_upper_bound_remove,
                                             sampling_lower_bound_add = sampling_lower_bound_add,
                                             sampling_upper_bound_add = sampling_upper_bound_add,
                                             batch_size = batch_size,
                                             bars = Constants.bars,
                                             samples_per_data_item = Constants.samples_per_ground_truth_data_item,
                                             beat_resolution = Constants.beat_resolution,
                                             beats_per_bar = Constants.beats_per_bar,
                                             number_of_pitches = Constants.number_of_pitches,
                                             number_of_channels = Constants.number_of_channels)

In [74]:
# Validation Data Generator
validation_data_generator = PianoRollGenerator(sample_list = validation_samples,
                                               sampling_lower_bound_remove = sampling_lower_bound_remove,
                                               sampling_upper_bound_remove = sampling_upper_bound_remove,
                                               sampling_lower_bound_add = sampling_lower_bound_add,
                                               sampling_upper_bound_add = sampling_upper_bound_add,
                                               batch_size = batch_size, 
                                               bars = Constants.bars,
                                               samples_per_data_item = Constants.samples_per_ground_truth_data_item,
                                               beat_resolution = Constants.beat_resolution,
                                               beats_per_bar = Constants.beats_per_bar, 
                                               number_of_pitches = Constants.number_of_pitches,
                                               number_of_channels = Constants.number_of_channels)

In [75]:
# Callback For Loss Plots 
plot_losses = GenerateTrainingPlots()
## Checkpoint Path
checkpoint_filepath =  'best-model-drums2.hdf5'

# Callback For Saving Model Checkpoints 
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

# Callback for logging
csv_logger = keras.callbacks.CSVLogger('training_log_drums2.csv', append=True, separator=',')

# Create A List Of Callbacks
callbacks_list = [plot_losses, model_checkpoint_callback, csv_logger]

In [76]:
# Create A Model Instance
MusicModel = ArCnnModel(input_dim = input_dim,
                        num_filters = num_filters,
                        growth_factor = growth_factor,
                        num_layers = num_layers,
                        dropout_rate_encoder = dropout_rate_encoder,
                        dropout_rate_decoder = dropout_rate_decoder,
                        batch_norm_encoder = batch_norm_encoder,
                        batch_norm_decoder = batch_norm_decoder,
                        pre_trained = pre_trained,
                        learning_rate = learning_rate,
                        optimizer_enum = optimizer_enum)

In [77]:
model = MusicModel.build_model()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 32) 320         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 32) 9248        conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 64, 64, 32)   0           conv2d_1[0][0]                   
______________________________________________________________________________________________

In [ ]:
# Resume from previous training instead if the file already exists
old_checkpoint_filepath =  'best-model-drums2.hdf5'
if os.path.isfile(old_checkpoint_filepath):
    model = load_model(old_checkpoint_filepath, 
                       custom_objects={'built_in_softmax_kl_loss': Loss.built_in_softmax_kl_loss})

In [ ]:
# Start Training
history = model.fit_generator(training_data_generator,
                              validation_data = validation_data_generator,
                              steps_per_epoch = steps_per_epoch,
                              epochs = epochs,
                              callbacks = callbacks_list)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/2
  9/246 [>.............................] - ETA: 1:45:11 - loss: 2.4225